### HYBparsimony

***HYBparsimony*** for Python is a package **for searching accurate parsimonious models by combining feature selection (FS), model hyperparameter optimization (HO), and parsimonious model selection (PMS) based on a separate cost and complexity evaluation** ([slices-HAIS2022](https://github.com/jodivaso/hybparsimony/blob/master/docs/presentacion_HAIS2022_javi.pdf), [slices-HAIS2023](https://github.com/jodivaso/hybparsimony/blob/master/docs/presentacion_HAIS2023_javi.pdf))

To improve the search for parsimony, the hybrid method combines GA mechanisms such as selection, crossover and mutation within a PSO-based optimization algorithm that includes a strategy in which the best position of each particle (thus also the best position of each neighborhood) is calculated taking into account not only the goodness-of-fit, but also the parsimony principle.

In HYBparsimony, the percentage of variables to be replaced with GA at each iteration $t$ is selected by a decreasing exponential function:
 $pcrossover=max(0.80 \cdot e^{(-\Gamma \cdot t)}, 0.10)$, that is adjusted by a $\Gamma$ parameter (by default $\Gamma$ is set to $0.50$). Thus, in the first iterations parsimony is promoted by GA mechanisms, i.e., replacing by crossover a high percentage of particles at the beginning. Subsequently, optimization with PSO becomes more relevant for the improvement of model accuracy. This differs from other hybrid methods in which the crossover is applied between the best individual position of each particle or other approaches in which the worst particles are also replaced by new particles, but at extreme positions.

[Experiments with 100 datasets](https://github.com/jodivaso/hybparsimony/tree/master/examples/analysis) showed that, in general, and with a suitable $\Gamma$, HYBparsimony allows to obtain better, more parsimonious and more robust models compared to other methods. It also reduces the number of iterations vs previous methods and, consequently, the computational effort.



### 1. Regression Examples

This example shows how to search with *hybparsimony* package for a parsimonious (with low complexity) *KernelRidge* with *rbf* kernel model and for the *diabetes* dataset. *HYBparsimony* searches for the best input features and *KernelRidge* hyperparameters: $alpha$ and $gamma$. Models are evaluated by default with a 5-fold CV negative mean squared error (*Neg MSE*). Finally, root mean squared error (*RMSE*) is calculated with another test dataset to check the degree of model generalization.

In this example, *rerank\_error* is set to $0.001$, but other values could improve the balance between model complexity and accuracy. PMS considers the most parsimonious model with the fewest number of features. The default complexity is $M_c = 10^9{N_{FS}} + int_{comp}$  where ${N_{FS}}$ is the number of selected input features and $int_{comp}$ is the internal measure of model complexity, which depends on the algorithm used for training. In this example, $int_{comp}$ for *KernelRidge* is measured by the sum of the squared coefficients. Therefore, between two models with the same number of features, the smaller sum of the squared weights will determine the more parsimonious model (smaller weights reduce the propagation of perturbations and improve robustness).

- **Note 1**: The datasets used in these examples are of small size to reduce elapsed times. These datasets have been selected in order to speed up the calculation process of the examples. With such small datasets it is necessary to use robust validation methods such as bootstrapping or repeated cross validation. It is also recommended to repeat the use of HYBparsimony with different random seeds in order to obtain more solid conclusions.
- **Note 2***: In some examples the number of iterations ('maxiter') has been reduced as well as the use of weak validation to minimize computation times. Due to these reasons some results could not be reliable. To obtain reliable results with these datasets, it is recommended to use repeated cross-validation and increase 'maxiter'.
- **Note 3**: The results of the examples may vary depending on the hardware available.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, RepeatedKFold
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_diabetes
from sklearn.preprocessing import StandardScaler
from hybparsimony import HYBparsimony
import os

#####################################################
#         Use sklearn regression algorithm          #
#####################################################

# Load 'diabetes' dataset
diabetes = load_diabetes()
X, y = diabetes.data, diabetes.target
print(X.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=1234)


# Standarize X and y (some algorithms require that)
scaler_X = StandardScaler()
X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)

scaler_y = StandardScaler()
y_train = scaler_y.fit_transform(y_train.reshape(-1,1)).flatten()
y_test = scaler_y.transform(y_test.reshape(-1,1)).flatten()
algo = 'KernelRidge'
HYBparsimony_model = HYBparsimony(algorithm=algo,
                                features=diabetes.feature_names,
                                rerank_error=0.001,
                                n_jobs=1, #Kernel Ridge uses all cores
                                verbose=1)
HYBparsimony_model.fit(X_train, y_train, time_limit=0.20)
# Check results with test dataset
preds = HYBparsimony_model.predict(X_test)

print(f'\n\nBest Model = {HYBparsimony_model.best_model}')
print(f'Selected features:{HYBparsimony_model.selected_features}')
print(f'Complexity = {round(HYBparsimony_model.best_complexity, 2):,}')
print(f'5-CV MSE = {-round(HYBparsimony_model.best_score,6)}')
print(f'RMSE test = {round(mean_squared_error(y_test, preds, squared=False),6)}')

(442, 10)
Detected a regression problem. Using 'neg_mean_squared_error' as default scoring function.
Running iteration 0
Best model -> Score = -0.510786 Complexity = 9,017,405,352.5 
Iter = 0 -> MeanVal = -0.88274  ValBest = -0.510786   ComplexBest = 9,017,405,352.5 Time(min) = 0.015769

Running iteration 1
Best model -> Score = -0.499005 Complexity = 8,000,032,783.88 
Iter = 1 -> MeanVal = -0.659969  ValBest = -0.499005   ComplexBest = 8,000,032,783.88 Time(min) = 0.014605

Running iteration 2
Best model -> Score = -0.498697 Complexity = 7,000,001,419.98 
Iter = 2 -> MeanVal = -0.784296  ValBest = -0.498697   ComplexBest = 7,000,001,419.98 Time(min) = 0.010077

Running iteration 3
Best model -> Score = -0.498697 Complexity = 7,000,001,419.98 
Iter = 3 -> MeanVal = -0.768574  ValBest = -0.498697   ComplexBest = 7,000,001,419.98 Time(min) = 0.004828

Running iteration 4
Best model -> Score = -0.492592 Complexity = 9,000,011,010.8 
Iter = 4 -> MeanVal = -0.726017  ValBest = -0.492592   C

Compare with different algorithms...

- ***Note:*** Increase *maxiter* to improve results

In [2]:
########################################################
#         Use different regression algorithms          #
########################################################
algorithms_reg = ['Ridge', 'Lasso', 'KernelRidge', 'KNeighborsRegressor', 'MLPRegressor', 'SVR',
'DecisionTreeRegressor', 'RandomForestRegressor']
res = []
for algo in algorithms_reg:
    print('#######################')
    print('Searching best: ', algo)
    HYBparsimony_model = HYBparsimony(algorithm=algo,
                                    features=diabetes.feature_names,
                                    maxiter=2, # 200 Extend to more iterations to improve results (time consuming)
                                    # cv=RepeatedKFold(n_splits=5, n_repeats=10), #uncomment to improve validation (time consuming)
                                    # n_jobs=20, # each job execute one fold
                                    rerank_error=0.001,
                                    verbose=1)
    # Search the best hyperparameters and features 
    # (increasing 'time_limit' to improve RMSE with high consuming algorithms)
    HYBparsimony_model.fit(X_train, y_train, time_limit=5)
    # Check results with test dataset
    preds = HYBparsimony_model.predict(X_test)
    print(algo, "RMSE test", mean_squared_error(y_test, preds, squared=False))
    print('Selected features:',HYBparsimony_model.selected_features)
    print(HYBparsimony_model.best_model)
    print('#######################')
    # Append results
    res.append(dict(algo=algo,
                    MSE_5CV= -round(HYBparsimony_model.best_score,6),
                    RMSE=round(mean_squared_error(y_test, preds, squared=False),6),
                    NFS=HYBparsimony_model.best_complexity//1e9,
                    selected_features = HYBparsimony_model.selected_features,
                    best_model=HYBparsimony_model.best_model))

res = pd.DataFrame(res).sort_values('RMSE')
# Visualize results
print(res[['best_model', 'MSE_5CV', 'RMSE', 'NFS', 'selected_features']])

#######################
Searching best:  Ridge
Detected a regression problem. Using 'neg_mean_squared_error' as default scoring function.
Running iteration 0


Best model -> Score = -0.49946 Complexity = 8,000,000,000.57 
Iter = 0 -> MeanVal = -0.596448   ValBest = -0.49946   ComplexBest = 8,000,000,000.57 Time(min) = 0.01322

Running iteration 1
Best model -> Score = -0.49946 Complexity = 8,000,000,000.57 
Iter = 1 -> MeanVal = -0.526338   ValBest = -0.49982   ComplexBest = 9,000,000,000.64 Time(min) = 0.004284

Early stopping reached. Stopped.
Ridge RMSE test 0.6954991719578381
Selected features: ['sex' 'bmi' 'bp' 's1' 's2' 's3' 's5' 's6']
Ridge(alpha=0.8391213911947143)
#######################
#######################
Searching best:  Lasso
Detected a regression problem. Using 'neg_mean_squared_error' as default scoring function.
Running iteration 0
Best model -> Score = -0.499801 Complexity = 9,000,000,000.52 
Iter = 0 -> MeanVal = -0.778465  ValBest = -0.499801   ComplexBest = 9,000,000,000.52 Time(min) = 0.004618

Running iteration 1
Best model -> Score = -0.499801 Complexity = 9,000,000,000.52 
Iter = 1 -> MeanVal = -0.593776  ValBest =

However, we can improve results in RMSE and parsimony if we increase the time limit to 60 minutes, the maximum number of iterations to 1000, and use a more robust validation with a 10-repeated 5-fold crossvalidation.

Example of code:
```python
HYBparsimony_model = HYBparsimony(algorithm=algo,
                                   features=diabetes.feature_names,
                                   rerank_error=0.001,
                                   cv=RepeatedKFold(n_repeats=10, n_splits=5),
                                   n_jobs=10, # each job executes one fold
                                   maxiter=1000,
                                   verbose=0)
HYBparsimony_model.fit(X_train, y_train, time_limit=60)
```

Note: *n_jobs* represents the number of CPU cores used within the *cross_val_score()* function included in *default_cv_score()*. Also, it is important to mention that certain *scikit-learn* algorithms inherently employ parallelization as well. Thus, with some algorithms it will be necessary to consider the sharing of cores between the algorithm and the cross_val_score() function.

The following table shows the best models found for each algorithm. In this case, **the model that best generalizes the problem is an ML regressor with only 6 features out of 10 and a single neuron in the hidden layer!**

|Algorithm|MSE\_10R5CV|RMSEtst|NFS|selected\_features|best\_model|
|-|-|-|-|-|-|
|**MLPRegressor**|0.493201|**0.671856**|**6**|['sex','bmi','bp','s1','s2','s5']|MLPRegressor(activation='logistic', alpha=0.010729877296924203, hidden_layer_sizes=1, max_iter=5000, n_iter_no_change=20, random_state=1234, solver='lbfgs', tol=1e-05)|\
|KernelRidge|0.483465|0.679036|7|['age','sex','bmi','bp','s3','s5','s6']|KernelRidge(alpha=0.3664462701238023, gamma=0.01808883688516421, kernel='rbf')|\
|SVR|0.487392|0.682699|8|['age','sex','bmi','bp','s1','s4','s5','s6']|SVR(C=0.8476135773996406, gamma=0.02324169209860404)|\
|KNeighborsRegressor|0.521326|0.687740|6|['sex','bmi','bp','s3','s5','s6']|KNeighborsRegressor(n\_neighbors=11)|\
|Lasso|0.493825|0.696194|7|['sex','bmi','bp','s1','s2','s5','s6']|Lasso(alpha=0.0002735058905983914)|\
|Ridge|0.492570|0.696273|7|['sex','bmi','bp','s1','s2','s5','s6']|Ridge(alpha=0.1321381563140431)|\
|RandomForestRegressor|0.552005|0.703769|9|['age','sex','bmi','bp','s2','s3','s4','s5','s6']|RandomForestRegressor(max_depth=17, min_samples_split=25, n_estimators=473)|\
|DecisionTreeRegressor|0.628316|0.864194|5|['age','sex','bmi','s4','s6']|DecisionTreeRegressor(max_depth=2, min_samples_split=20)|\

2. Binary Classification

This example shows how to use *HYBparsimony* in a binary classification problem with *breast_cancer* dataset (30 inputs). By default, method uses *LogisticRegression* algorithm and *neg_log_loss* as scoring metric.

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import log_loss
from hybparsimony import HYBparsimony

# load 'breast_cancer' dataset
breast_cancer = load_breast_cancer()
X, y = breast_cancer.data, breast_cancer.target 
print(X.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=1)

# Standarize X and y (some algorithms require that)
scaler_X = StandardScaler()
X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)

HYBparsimony_model = HYBparsimony(features=breast_cancer.feature_names,
                                rerank_error=0.005,
                                verbose=1)
HYBparsimony_model.fit(X_train, y_train, time_limit=0.50)
# Extract probs of class==1
preds = HYBparsimony_model.predict_proba(X_test)[:,1]
print(f'\n\nBest Model = {HYBparsimony_model.best_model}')
print(f'Selected features:{HYBparsimony_model.selected_features}')
print(f'Complexity = {round(HYBparsimony_model.best_complexity, 2):,}')
print(f'5-CV logloss = {-round(HYBparsimony_model.best_score,6)}')
print(f'logloss test = {round(log_loss(y_test, preds),6)}')

(569, 30)
Detected a binary-class problem. Using 'neg_log_loss' as default scoring function.
Running iteration 0
Best model -> Score = -0.091519 Complexity = 29,000,000,005.11 
Iter = 0 -> MeanVal = -0.297449  ValBest = -0.091519   ComplexBest = 29,000,000,005.11 Time(min) = 0.016088

Running iteration 1
Best model -> Score = -0.085673 Complexity = 27,000,000,009.97 
Iter = 1 -> MeanVal = -0.117216  ValBest = -0.085673   ComplexBest = 27,000,000,009.97 Time(min) = 0.010721

Running iteration 2
Best model -> Score = -0.08542 Complexity = 14,000,000,024.34 
Iter = 2 -> MeanVal = -0.122737   ValBest = -0.08542   ComplexBest = 14,000,000,024.34 Time(min) = 0.010556

Running iteration 3
Best model -> Score = -0.085181 Complexity = 22,000,000,017.16 
Iter = 3 -> MeanVal = -0.11499  ValBest = -0.085181   ComplexBest = 22,000,000,017.16 Time(min) = 0.011141

Running iteration 4
Best model -> Score = -0.085181 Complexity = 22,000,000,017.16 
Iter = 4 -> MeanVal = -0.117198  ValBest = -0.086371 

Search the best model with different algorithms...

- ***Note:*** Increase *maxiter* to improve results

In [4]:
algorithms_clas = ['LogisticRegression', 'MLPClassifier', 
                        'SVC', 'DecisionTreeClassifier',
                        'RandomForestClassifier', 'KNeighborsClassifier',
                        ]
res = []
for algo in algorithms_clas:
    print('#######################')
    print('Searching best: ', algo)
    HYBparsimony_model = HYBparsimony(algorithm=algo,
                                    features=breast_cancer.feature_names,
                                    rerank_error=0.005,
                                    maxiter=2, # extend to more iterations (time consuming)
                                    # cv=RepeatedKFold(n_splits=5, n_repeats=10), #uncomment to improve validation (time consuming)
                                    # n_jobs=20, # each job executes one fold
                                    verbose=1)
    # Search the best hyperparameters and features 
    # (increasing 'time_limit' to improve neg_log_loss with high consuming algorithms)
    HYBparsimony_model.fit(X_train, y_train, time_limit=60)
    # Check results with test dataset
    preds = HYBparsimony_model.predict_proba(X_test)[:,1]
    print(algo, "Logloss_Test=", round(log_loss(y_test, preds),6))
    print('Selected features:',HYBparsimony_model.selected_features)
    print(HYBparsimony_model.best_model)
    print('#######################')
    # Append results
    res.append(dict(algo=algo,
                    Logloss_10R5CV= -round(HYBparsimony_model.best_score,6),
                    Logloss_Test = round(log_loss(y_test, preds),6),
                    NFS=int(HYBparsimony_model.best_complexity//1e9),
                    selected_features = HYBparsimony_model.selected_features,
                    best_model=HYBparsimony_model.best_model))
res = pd.DataFrame(res).sort_values('Logloss_Test')
# Visualize results
print(res[['algo', 'Logloss_10R5CV', 'Logloss_Test', 'NFS']])

#######################
Searching best:  LogisticRegression
Detected a binary-class problem. Using 'neg_log_loss' as default scoring function.
Running iteration 0
Best model -> Score = -0.091519 Complexity = 29,000,000,005.11 
Iter = 0 -> MeanVal = -0.297449  ValBest = -0.091519   ComplexBest = 29,000,000,005.11 Time(min) = 0.012875

Running iteration 1
Best model -> Score = -0.085673 Complexity = 27,000,000,009.97 
Iter = 1 -> MeanVal = -0.117216  ValBest = -0.085673   ComplexBest = 27,000,000,009.97 Time(min) = 0.009914

LogisticRegression Logloss_Test= 0.080637
Selected features: ['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concave points error'
 'symmetry error' 'fractal dimension error' 'worst radius'
 'worst perimeter' 'worst area' 'worst smoothne

### 3. Multiclass Classification

If the number of classes is greather than 2, *HYBparsimony* selects *f1\_macro* as scoring metric. In this example, we increase the number of particles to 20 with $npart=20$ and the $time\\_limit$ to 5 minutes. However, we also include an early stopping if best individual does not change in 20 iterations $early\\_stop=20$

In [5]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split, RepeatedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_wine
from sklearn.metrics import f1_score
from hybparsimony import HYBparsimony

# load 'wine' dataset 
wine = load_wine()
X, y = wine.data, wine.target 
print(X.shape)
# 3 classes
print(len(np.unique(y)))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=1)

# Standarize X and y (some algorithms require that)
scaler_X = StandardScaler()
X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)

HYBparsimony_model = HYBparsimony(features=wine.feature_names,
                                cv=RepeatedKFold(n_splits=5, n_repeats=10),
                                npart = 20,
                                early_stop=20,
                                rerank_error=0.001,
                                n_jobs=10, #Use 10 cores (1 core runs 1 fold)
                                verbose=1)
HYBparsimony_model.fit(X_train, y_train, time_limit=5.0)
preds = HYBparsimony_model.predict(X_test)
print(f'\n\nBest Model = {HYBparsimony_model.best_model}')
print(f'Selected features:{HYBparsimony_model.selected_features}')
print(f'Complexity = {round(HYBparsimony_model.best_complexity, 2):,}')
print(f'10R5-CV f1_macro = {round(HYBparsimony_model.best_score,6)}')
print(f'f1_macro test = {round(f1_score(y_test, preds, average="macro"),6)}')

(178, 13)
3
Detected a multi-class problem. Using 'f1_macro' as default scoring function.
Running iteration 0
Best model -> Score = 0.981068 Complexity = 13,000,000,001.38 
Iter = 0 -> MeanVal = 0.759953   ValBest = 0.981068   ComplexBest = 13,000,000,001.38 Time(min) = 0.063593

Running iteration 1
Best model -> Score = 0.985503 Complexity = 11,000,000,036.33 
Iter = 1 -> MeanVal = 0.938299   ValBest = 0.985503   ComplexBest = 11,000,000,036.33 Time(min) = 0.03999

Running iteration 2
Best model -> Score = 0.98892 Complexity = 11,000,000,038.25 
Iter = 2 -> MeanVal = 0.963618   ValBest = 0.98892    ComplexBest = 11,000,000,038.25 Time(min) = 0.03825

Running iteration 3
Best model -> Score = 0.98958 Complexity = 7,000,000,344.54 
Iter = 3 -> MeanVal = 0.965505   ValBest = 0.98958    ComplexBest = 7,000,000,344.54 Time(min) = 0.039044

Running iteration 4
Best model -> Score = 0.98958 Complexity = 7,000,000,344.54 
Iter = 4 -> MeanVal = 0.964892   ValBest = 0.989042   ComplexBest = 9,0

### 4. Custom Evaluation

*HYBparsimony* uses by default sklearn's [*cross_val_score*](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) function as follows:

```python
def default_cv_score(estimator, X, y):
  return cross_val_score(estimator, X, y, cv=self.cv, scoring=self.scoring)
```

By default $cv=5$, and $scoring$ is defined as *MSE* for regression problems, *log_loss* for binary classification problems, and *f1_macro* for multiclass problems. However, it is possible to choose [another scoring metric](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter) defined in *scikit-learn* library or design [your own](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring). Also, the user can define a custom evaluation function.

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_breast_cancer, load_wine
from sklearn.model_selection import cross_val_score, RepeatedKFold
from hybparsimony import HYBparsimony
from sklearn.metrics import fbeta_score, make_scorer, cohen_kappa_score, log_loss, accuracy_score
import os

# load 'breast_cancer' dataset
breast_cancer = load_breast_cancer()
X, y = breast_cancer.data, breast_cancer.target 
print(X.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=1)

# Standarize X and y (some algorithms require that)
scaler_X = StandardScaler()
X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)

(569, 30)


#### 4.1 Example A: Using 10 folds and 'accuracy'

In [7]:
HYBparsimony_model = HYBparsimony(features=breast_cancer.feature_names,
                                    scoring='accuracy',
                                    cv=10,
                                    n_jobs=10, #Use 10 cores (1 core run 1 fold)
                                    rerank_error=0.001,
                                    verbose=1)

HYBparsimony_model.fit(X_train, y_train, time_limit=0.1)
preds = HYBparsimony_model.predict(X_test)
print(f'\n\nBest Model = {HYBparsimony_model.best_model}')
print(f'Selected features:{HYBparsimony_model.selected_features}')
print(f'Complexity = {round(HYBparsimony_model.best_complexity, 2):,}')
print(f'10R5-CV Accuracy = {round(HYBparsimony_model.best_score,6)}')
print(f'Accuracy test = {round(accuracy_score(y_test, preds),6)}')

Using 'accuracy' as scoring function.
Running iteration 0
Best model -> Score = 0.975845 Complexity = 29,000,000,005.11 
Iter = 0 -> MeanVal = 0.889407   ValBest = 0.975845   ComplexBest = 29,000,000,005.11 Time(min) = 0.021194

Running iteration 1
Best model -> Score = 0.980338 Complexity = 25,000,000,009.16 
Iter = 1 -> MeanVal = 0.96771   ValBest = 0.980338   ComplexBest = 25,000,000,009.16 Time(min) = 0.016287

Running iteration 2
Best model -> Score = 0.980338 Complexity = 25,000,000,009.16 
Iter = 2 -> MeanVal = 0.96609   ValBest = 0.978116   ComplexBest = 14,000,000,047.55 Time(min) = 0.014893

Running iteration 3
Best model -> Score = 0.980338 Complexity = 25,000,000,009.16 
Iter = 3 -> MeanVal = 0.962306   ValBest = 0.978116   ComplexBest = 22,000,000,011.33 Time(min) = 0.012363

Running iteration 4
Best model -> Score = 0.980338 Complexity = 25,000,000,009.16 
Iter = 4 -> MeanVal = 0.969018   ValBest = 0.98029    ComplexBest = 21,000,000,012.79 Time(min) = 0.012556

Running i

#### 4.2 Example B: Using 10-repeated 5-fold CV and 'Kappa' score

In [8]:
# load 'wine' dataset 
wine = load_wine()
X, y = wine.data, wine.target 
print(X.shape)
# 3 classes
print(len(np.unique(y)))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=1)

# Standarize X and y (some algorithms require that)
scaler_X = StandardScaler()
X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)


metric_kappa = make_scorer(cohen_kappa_score, greater_is_better=True)
HYBparsimony_model = HYBparsimony(features=wine.feature_names,
                                scoring=metric_kappa,
                                cv=RepeatedKFold(n_splits=5, n_repeats=10),
                                n_jobs=10, #Use 10 cores (one core=one fold)
                                rerank_error=0.001,
                                verbose=1)
HYBparsimony_model.fit(X_train, y_train, time_limit=0.1)
print(f'\n\nBest Model = {HYBparsimony_model.best_model}')
print(f'Selected features:{HYBparsimony_model.selected_features}')

(178, 13)
3
Using 'make_scorer(cohen_kappa_score)' as scoring function.
Running iteration 0
Best model -> Score = 0.962774 Complexity = 13,000,000,009.17 
Iter = 0 -> MeanVal = 0.706077   ValBest = 0.962774   ComplexBest = 13,000,000,009.17 Time(min) = 0.029052

Running iteration 1
Best model -> Score = 0.9719 Complexity = 13,000,000,002.52 
Iter = 1 -> MeanVal = 0.939917    ValBest = 0.9719    ComplexBest = 13,000,000,002.52 Time(min) = 0.029599

Running iteration 2
Best model -> Score = 0.972549 Complexity = 9,000,000,026.48 
Iter = 2 -> MeanVal = 0.942237   ValBest = 0.972549   ComplexBest = 9,000,000,026.48 Time(min) = 0.02718

Running iteration 3
Time limit reached. Stopped.


Best Model = LogisticRegression(C=3.136879012883974)
Selected features:['alcohol' 'malic_acid' 'ash' 'flavanoids' 'nonflavanoid_phenols'
 'proanthocyanins' 'hue' 'od280/od315_of_diluted_wines' 'proline']


#### 4.3 Example C: Using a weighted 'log_loss'

In [9]:
# Assign a double weight to class one
def my_custom_loss_func(y_true, y_pred):
    sample_weight = np.ones_like(y_true)
    sample_weight[y_true==1] = 2.0
    return log_loss(y_true, y_pred, sample_weight=sample_weight)

# load 'breast_cancer' dataset
breast_cancer = load_breast_cancer()
X, y = breast_cancer.data, breast_cancer.target 
print(X.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=1)

# Standarize X and y (some algorithms require that)
scaler_X = StandardScaler()
X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)

# Lower is better and 'log_loss' needs probabilities
custom_score = make_scorer(my_custom_loss_func, greater_is_better=False, needs_proba=True)
HYBparsimony_model = HYBparsimony(features=breast_cancer.feature_names,
                                scoring=custom_score,
                                rerank_error=0.001,
                                verbose=1)
HYBparsimony_model.fit(X_train, y_train, time_limit=0.20)
print(f'\n\nBest Model = {HYBparsimony_model.best_model}')
print(f'Selected features:{HYBparsimony_model.selected_features}')

(569, 30)
Using 'make_scorer(my_custom_loss_func, greater_is_better=False, needs_proba=True)' as scoring function.
Running iteration 0
Best model -> Score = -0.078242 Complexity = 24,000,000,013.5 
Iter = 0 -> MeanVal = -0.257253  ValBest = -0.078242   ComplexBest = 24,000,000,013.5 Time(min) = 0.036192

Running iteration 1
Best model -> Score = -0.073602 Complexity = 25,000,000,009.77 
Iter = 1 -> MeanVal = -0.102049  ValBest = -0.073602   ComplexBest = 25,000,000,009.77 Time(min) = 0.015747

Running iteration 2
Best model -> Score = -0.073134 Complexity = 23,000,000,014.89 
Iter = 2 -> MeanVal = -0.098238  ValBest = -0.073134   ComplexBest = 23,000,000,014.89 Time(min) = 0.013923

Running iteration 3
Best model -> Score = -0.069526 Complexity = 17,000,000,024.25 
Iter = 3 -> MeanVal = -0.101725  ValBest = -0.069526   ComplexBest = 17,000,000,024.25 Time(min) = 0.015503

Running iteration 4
Best model -> Score = -0.069526 Complexity = 17,000,000,024.25 
Iter = 4 -> MeanVal = -0.08713 

#### 4.4 Example D: Using a 'custom evaluation' function 

In [10]:
def custom_fun(estimator, X, y):
    return cross_val_score(estimator, X, y, scoring="accuracy", n_jobs=10)

HYBparsimony_model = HYBparsimony(features=breast_cancer.feature_names,
                                custom_eval_fun=custom_fun,
                                rerank_error=0.001,
                                verbose=1)


HYBparsimony_model.fit(X_train, y_train, time_limit=0.20)
preds = HYBparsimony_model.predict(X_test)
print(f'\n\nBest Model = {HYBparsimony_model.best_model}')
print(f'Selected features:{HYBparsimony_model.selected_features}')
print(f'Complexity = {round(HYBparsimony_model.best_complexity, 2):,}')
print(f'10R5-CV Accuracy = {round(HYBparsimony_model.best_score,6)}')
print(f'Accuracy test = {round(accuracy_score(y_test, preds),6)}')

Running iteration 0
Best model -> Score = 0.975824 Complexity = 29,000,000,005.11 
Iter = 0 -> MeanVal = 0.884835   ValBest = 0.975824   ComplexBest = 29,000,000,005.11 Time(min) = 0.03483

Running iteration 1
Best model -> Score = 0.982418 Complexity = 25,000,000,013.27 
Iter = 1 -> MeanVal = 0.965714   ValBest = 0.982418   ComplexBest = 25,000,000,013.27 Time(min) = 0.013752

Running iteration 2
Best model -> Score = 0.982418 Complexity = 25,000,000,013.27 
Iter = 2 -> MeanVal = 0.966154   ValBest = 0.973626   ComplexBest = 20,000,000,012.68 Time(min) = 0.01234

Running iteration 3
Best model -> Score = 0.982418 Complexity = 25,000,000,013.27 
Iter = 3 -> MeanVal = 0.966007   ValBest = 0.978022   ComplexBest = 20,000,000,013.53 Time(min) = 0.010652

Running iteration 4
Best model -> Score = 0.982418 Complexity = 25,000,000,013.27 
Iter = 4 -> MeanVal = 0.963956   ValBest = 0.978022   ComplexBest = 16,000,000,022.01 Time(min) = 0.00968

Running iteration 5
Best model -> Score = 0.9824

### 5. Custom Search

*HYBparsimony* has predefined the most common scikit-learn algorithms as well as functions to measure [their complexity](https://github.com/jodivaso/hybparsimony/blob/master/hybparsimony/util/complexity.py) and the [hyperparameter ranges](https://github.com/jodivaso/hybparsimony/blob/master/hybparsimony/util/models.py) to search on. However, all this can be customized. 

In the following example, the dictionary *MLPRegressor_new* is defined. It consists of the following properties:
- *estimator* any machine learning algorithm compatible with scikit-learn.
- *complexity* the function that measures the complexity of the model.
- The hyperparameters of the algorithm. In this case, they can be fixed values (defined by Population.CONSTANT) such as '*solver*', '*activation*', etc.; or a search range $[min, max]$ defined by *{"range":(min, max), "type": Population.X}* and which type can be of three values: integer (Population.INTEGER), float (Population.FLOAT) or in powers of 10 (Population.POWER), i.e. $10^{[min, max]}$.

In [11]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split, RepeatedKFold
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_diabetes
from sklearn.preprocessing import StandardScaler
from hybparsimony import HYBparsimony, Population


# Load 'diabetes' dataset
diabetes = load_diabetes()

X, y = diabetes.data, diabetes.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=1234)

# Standarize X and y
scaler_X = StandardScaler()
X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)
scaler_y = StandardScaler()
y_train = scaler_y.fit_transform(y_train.reshape(-1,1)).flatten()
y_test = scaler_y.transform(y_test.reshape(-1,1)).flatten()

def mlp_new_complexity(model, nFeatures, **kwargs):
    weights = [np.concatenate(model.intercepts_)]
    for wm in model.coefs_:
        weights.append(wm.flatten())
    weights = np.concatenate(weights) 
    int_comp = np.min((1E09-1,np.sum(weights**2)))
    return nFeatures*1E09 + int_comp

MLPRegressor_new = {"estimator": MLPRegressor, # The estimator
                "complexity": mlp_new_complexity, # The complexity
                "hidden_layer_sizes": {"range": (1, 5), "type": Population.INTEGER},
                "alpha": {"range": (-5, 5), "type": Population.POWER},
                "solver": {"value": "adam", "type": Population.CONSTANT},
                "learning_rate": {"value": "adaptive", "type": Population.CONSTANT},
                "early_stopping": {"value": True, "type": Population.CONSTANT},
                "validation_fraction": {"value": 0.10, "type": Population.CONSTANT},
                "activation": {"value": "tanh", "type": Population.CONSTANT},
                "n_iter_no_change": {"value": 20, "type": Population.CONSTANT},
                "tol": {"value": 1e-5, "type": Population.CONSTANT},
                "random_state": {"value": 1234, "type": Population.CONSTANT},
                "max_iter": {"value": 200, "type": Population.CONSTANT}
                }
HYBparsimony_model = HYBparsimony(algorithm=MLPRegressor_new,
                                features=diabetes.feature_names,
                                cv=RepeatedKFold(n_splits=5, n_repeats=10),
                                n_jobs= 25, #Use 25 cores (one core=one fold)
                                maxiter=2, # Extend to more generations (time consuming)
                                npart = 10,
                                rerank_error=0.001,
                                verbose=1)

# Search the best hyperparameters and features 
# (increasing 'time_limit' to improve RMSE with high consuming algorithms)
HYBparsimony_model.fit(X_train, y_train, time_limit=1.00)
preds = HYBparsimony_model.predict(X_test)
print(f'\n\nBest Model = {HYBparsimony_model.best_model}')
print(f'Selected features:{HYBparsimony_model.selected_features}')
print(f'Complexity = {round(HYBparsimony_model.best_complexity, 2):,}')
print(f'5-CV MSE = {-round(HYBparsimony_model.best_score,6)}')
print(f'RMSE test = {round(mean_squared_error(y_test, preds, squared=False),6)}')

Detected a regression problem. Using 'neg_mean_squared_error' as default scoring function.
Running iteration 0
Best model -> Score = -0.553939 Complexity = 10,000,000,006.93 
Iter = 0 -> MeanVal = -0.881378  ValBest = -0.553939   ComplexBest = 10,000,000,006.93 Time(min) = 0.096929

Running iteration 1
Best model -> Score = -0.553939 Complexity = 10,000,000,006.93 
Iter = 1 -> MeanVal = -0.665962  ValBest = -0.559135   ComplexBest = 9,000,000,005.57 Time(min) = 0.079647

Early stopping reached. Stopped.


Best Model = MLPRegressor(activation='tanh', alpha=0.00023061450085985692,
             early_stopping=True, hidden_layer_sizes=4,
             learning_rate='adaptive', n_iter_no_change=20, random_state=1234,
             tol=1e-05)
Selected features:['age' 'sex' 'bmi' 'bp' 's1' 's2' 's3' 's4' 's5' 's6']
Complexity = 10,000,000,006.93
5-CV MSE = 0.553939
RMSE test = 0.767647


### 6. Using AutoGluon

[This notebook](https://github.com/jodivaso/hybparsimony/blob/master/examples/Autogluon_with_SHDD.ipynb) shows **how to reduce the input features from 85 to 44 (51.7%)** of an [AutoGluon](https://auto.gluon.ai/stable/index.html) model for the COIL2000 dataset downloaded from [openml.com](https://www.openml.org/). The difference in the 'log_loss' (with a test dataset) of the model trained with the 85 features versus the 44 features **is only 0.000312**.

### References

Divas�n, J., Pernia-Espinoza, A., Martinez-de-Pison, F.J. (2023). [HYB-PARSIMONY: A hybrid approach combining Particle Swarm Optimization and Genetic Algorithms to find parsimonious models in high-dimensional datasets](https://authors.elsevier.com/sd/article/S0925-2312(23)00963-3). Neurocomputing, 560, 126840.
2023, Elsevier. [https://doi.org/10.1016/j.neucom.2023.126840](https://doi.org/10.1016/j.neucom.2023.126840)